In [4]:
import re
import os
import math
import random
#from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from numpy import dot
from numpy.linalg import norm
import pandas as pd
import operator

In [5]:
def read_file():

    collection = {}
    docId = 1
    classes = []

    for folder in os.listdir("C:/Users/Areeka Aijaz/Desktop/IR Assignment 3/bbcsport"):

        N = 0
        
        if not folder.endswith(".TXT"):
            folderPath = 'C:/Users/Areeka Aijaz/Desktop/IR Assignment 3/bbcsport/' + folder

            for document in os.listdir(folderPath):
                N += 1
                collection[docId] = []
                docPath = folderPath + '/' + document
                tokens = tokenize(docPath)
                collection[docId].extend(tokens)
                docId += 1

            classes.append((folder,N))
                
    return len(collection),collection,classes

In [6]:
def tokenize(docPath):
    
    symbols = ['$','[',']','.','?',';',':','-','!','--',',','...','0','1','2','3','4','5','6','7','8','9','\\','"','(',')','{','}','%','&','/','',"'re","n't","'s","'ve","'d","'m"]

    nt_symbols = ['-','.',']',"n't",'--','...']
    
    tokens = []
    
    flag = False
    
    doc = open(docPath,'r')

    for line in doc:
        
        for word in line.split():
            
            for sym in symbols:
                if sym in word and sym not in nt_symbols:
                    word = word.replace(sym,'')

            if word != '' and word[len(word)-1] == '.':
                word = word.replace('.','')

            if '-' in word :
                word = re.split(r'[-.]',word)
                flag = True

            if flag:
                tokens.extend(word)
                flag = False
            else:
                tokens.append(word)    
            
    preProcessedTokens = pre_processing(tokens)
            
    doc.close()
    
    return preProcessedTokens

In [7]:
def pre_processing(tokens):
    
    stopWordsList = ['a','is','the','of','all','and','to','can','be','as','once','for','at','am','are','have','had','up','his','her','in','on','we','do','']
    
    preProcessedToken = []
    
    ps = PorterStemmer()
    
    #lemm = WordNetLemmatizer()
    
    for word in tokens:
        if word.casefold() not in stopWordsList:
            word = word.casefold()
            word = ps.stem(word)
            #word = lemm.lemmatize(word)
            preProcessedToken.append(word)
        
    return preProcessedToken

In [8]:
def make_dictionary(collection):

    dictionary = []

    for i in collection:
        for term in collection[i] :
            if term not in dictionary and term.isalpha() and len(term)>2:
                dictionary.append(term)

    dictionary.sort()
    
    return dictionary 

In [9]:
def calculate_tf(collection, dictionary):
    
    termFrequency = {}

    for token in dictionary:
        termFrequency[token] = []

        for i in collection:
            termFrequency[token].append(collection[i].count(token))
    
    return termFrequency

In [10]:
def calculate_df(termFrequency, N):

    documentFrequency = {}

    for term in termFrequency:   
        documentFrequency[term] = 0

        for count in termFrequency[term]:
            if count > 0:
                documentFrequency[term] += 1
                
    return documentFrequency

In [11]:
def feature_selection(dict_, tf, df):
    
    dictionary = []
    termFrequency = {}
    documentFrequency = {}
    
    for i in range(len(dict_)):
        if df[dict_[i]]>5 :
            dictionary.append(dict_[i])
            
            documentFrequency[dict_[i]] = df[dict_[i]]
                              
            termFrequency[dict_[i]] = tf[dict_[i]]
    
    return dictionary, termFrequency, documentFrequency

In [12]:
def calculate_idf(documentFrequency, N):

    idf = {}

    for term in documentFrequency:  
        idf[term] = math.log10(N/documentFrequency[term])
        
    return idf

In [13]:
def tf_idf_scoring(termFrequency, idf):

    docScore = {}

    for term in termFrequency:
        docScore[term] = []

        for tf in termFrequency[term]:
            docScore[term].append(tf*idf[term])

    return docScore

In [14]:
"""
documentVector = [
    doc1 = ([score[term1],score[term2],...,score[term n]], class a)
    doc2 = ([score[term1],score[term2],...,score[term n]], class b)
    .....
    docN = ([score[term1],score[term2],...,score[term n]], class x)
]
"""

def get_document_vector(N,docScore,classes):
    
    documentVector = []
    k = 0
    j = classes[0][1]
    
    for i in range(N):
        document = []
        
        for term in docScore:
            document.append(docScore[term][i])
        
        if i < j :
            documentVector.append((document,classes[k][0]))
        else:
            j = classes[k+1][1] + i
            k += 1
            documentVector.append((document,classes[k][0]))
        
    return documentVector

In [19]:
def K_means_algorithm(documentVector, classes, features):
    
    K = len(classes)
    N = len(documentVector)
    
    cluster = {}
    
    #Selecting random seed----------------------(by taking minimum cosine similarity to achieve maximum difference between clusters)
    
    centroid = []
    
    randomDoc = random.sample(range(0, N), K)
    
    for i in range(len(randomDoc)):
        temp = []
        for j in range(len(documentVector)):
            temp.append(dot(documentVector[randomDoc[i]][0], documentVector[j][0])/(norm(documentVector[randomDoc[i]][0])*norm(documentVector[j][0])))
            
        docNo , minSimilarity = min(enumerate(temp), key = operator.itemgetter(1))
        
        centroid.append(documentVector[docNo][0])
        
    #--------------------------------------------
    
    x = 0
    while (x<200):
        
        tempCluster = {}
        
        for i in range(K):
            tempCluster['c'+ str(i+1)] = []
        
        for i in range(N):
            similarity = []

            for j in range(K):
                similarity.append(dot(centroid[j], documentVector[i][0])/(norm(centroid[j])*norm(documentVector[i][0])))

            clusterNo , maxSimilarity = max(enumerate(similarity), key = operator.itemgetter(1))

            tempCluster['c' + str(clusterNo+1)].append(documentVector[i])
            
        for i in tempCluster:
            sum_ = 0
            temp = []

            for j in range(features):
                for k in tempCluster[i]:
                    sum_ += k[0][j]

                sum_ /= len(tempCluster[i])
                temp.append(sum_)

            centroid.append(temp)
        
        x += 1
        
    return tempCluster

In [ ]:
N,collection,classes = read_file()
dictionary = make_dictionary(collection)
termFrequency = calculate_tf(collection,dictionary)
documentFrequency = calculate_df(termFrequency, N)
dictionary, termFrequency, documentFrequency = feature_selection(dictionary, termFrequency, documentFrequency)
idf = calculate_idf(documentFrequency,N)
docScore = tf_idf_scoring(termFrequency, idf)
documentVector = get_document_vector(N,docScore,classes)
features = len(dictionary)
clusters = K_means_algorithm(documentVector, classes, features)

In [21]:
for i in clusters:
    print(len(clusters[i]))
    for j in clusters[i]:
        print(i,j[1])

149
c1 athletics
c1 athletics
c1 athletics
c1 athletics
c1 athletics
c1 athletics
c1 athletics
c1 athletics
c1 athletics
c1 athletics
c1 athletics
c1 athletics
c1 athletics
c1 athletics
c1 athletics
c1 athletics
c1 athletics
c1 athletics
c1 athletics
c1 cricket
c1 cricket
c1 cricket
c1 cricket
c1 cricket
c1 cricket
c1 cricket
c1 cricket
c1 cricket
c1 cricket
c1 cricket
c1 cricket
c1 cricket
c1 cricket
c1 cricket
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 football
c1 footb